Давайте построим простую классификацию для пассажиров Титаника

По данным пассажира будет пытаться угадать, выжил он или нет

В этом ноутбуке сделаем предобработку тренировочных данных, а построение классификатора и вычисление метрик качества на тестовых данных будут домашним заданием

Вам понадобятся датасеты Titanic_train.csv и Titanic_test.csv

In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB

In [2]:
# Считаем тренировочный датасет
df = pd.read_csv('Titanic_train.csv')

In [ ]:
# посмотрим, как обращаться к колонкам датасета
df.columns

В первом блоке анализа мы посмотрим, какие у нас есть данные, какие данные пропущены

In [ ]:
# полюбуемся на фрагмент датасета
df.sample(5)

In [ ]:
# выясним, в каких столбцах не хватает данных
df.isna().sum()

Данных не хватает в столбцах Age, Cabin, Embarked:
    1. У 143 пассажиров не известен возраст
    2. у 552 пассажиров не известен номер каюты
    3. У 2 пассажиров не известен порт посадки

In [3]:
# Недостающие данные по возрасту заполним средним арифметическим
mean_age = df['Age'].mean() # здесь вычисляем
print ('средний возраст пассажира: ',mean_age)
df['Age'].fillna(mean_age , inplace=True) # здесь заполняем

средний возраст пассажира:  29.539402460456945


In [ ]:
# Посмотрим, из какого порта отправилось большинство пассажиров
# будет логично предположить, что 2 пассажира с неизвестным портом отправления сели там же
print(df['Embarked'].value_counts())

In [4]:
# Больше всего пассажиров отправились в путь на Титанике из порта S - Southampton
# Назначим это значение пассажирам, чей порт посадки не известен
df['Embarked'].fillna('S',inplace=True)

В столбцах Unnamed: 0' и 'PassengerId содержатся порядковые номера пассажиров, они нам не нужны

'Cabin' содержит слишком много пропущенных значений, заполнять их бессмысленно, хотя
из этих данных, вернее, из их наличия можно было бы извлечь полезную информацию: единственный список с номерами кают есть был найден только для пассажиров 1 класса , а вот у пассажиров 2 и 3 класса наличие номера каюты означает, что этот пассажир, скорее всего выжил. Это может стать точкой роста для классификатора, но сейчас мы все равно избавимся от этого столбца

Из 'Name' и 'Ticket' (идентификатор билета) тоже можно вытащить полезную информацию, однако сейчас опять же эти данные рассматривать не будем

In [5]:
# удаляем лишние столбцы из датасета
del df['Unnamed: 0']
del df['PassengerId']
del df['Name']
del df['Cabin']
del df['Ticket']

In [6]:
#  сделаем предобработку данных по полу и порту посадки
# они заданы как категории, нам же для обучения нужны числа
# есть разные способы превратить категории в числа, один из них 
# - задать числовые категории по словарю

# создаем словари
dict_Sex = {'male': 1, 'female': 0}
dict_Embarked = {'S': 0, 'C': 1, 'Q':2}

# преобразуем данные при помощи map
df['Sex'] = df ['Sex'].map(dict_Sex )
df['Embarked'] = df ['Embarked'].map(dict_Embarked )



In [7]:
# полюбуемся на то, что у нас получилось
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,47.000000,0,0,9.0000,0
1,1,3,1,29.539402,0,0,7.7500,2
2,0,2,1,34.000000,1,0,26.0000,0
3,1,1,0,44.000000,0,1,57.9792,1
4,0,3,0,47.000000,1,0,14.5000,0


In [ ]:
# разделим данные на признаки и целевую переменную
x_train = df.drop(['Survived'], axis = 1)
y_train = df[['Survived']]

Ваше задание:
1. Сделать точно такую же предобработку тестовых данных. Подумайте, на какие значения будет корректно заменять пропуски в графах "возраст" и "порт посадки" в тестовой выборке.
2. Обучить НБК. Какую модель вы выберете?
3. Вычислить confusion matrix на тестовых данных
4. Вычислить метрики качества модели
5. Подумайте, каких пассажиров модель классифицирует лучше - выживших или не выживших
6. Подумайте, как проинтерпретировать ошибку второго рода